# <center>Ejemplo Contenedores de Datos</center>

En este cuaderno realizaremos un programa que lleve el registro de Ingredientes para recetas de cocina. Dentro del sistema podremos registrar ingredientes, consultarlos y buscarlos por nombre. Para este ejemplo utilizaremos contenedores de datos, en particular usaremos arreglo, registro, unión y enumerado.

Comenzaremos incluyendo la biblioteca estándar para flujos de entrada y salida así como el espacio de nombres estándar.

In [ ]:
#include <iostream>
using namespace std;

Los ingredientes serán registrados con una cantidad, sin embargo, no es lo mismo intentar medir harina que intentar medir aceite, el primero suele medirse en gramos o kilos mientras que el segundo comunmente se mide en mililitros o litros. Para dar más sentido a esa variable que represente la cantidad necesaria del ingrediente utilizaremos una unión. 

En nuestro caso podremos representar la medida como un entero si queremos registrarla como gramos o mililitros y como un flotante si quisiéramos registrar kilos o litros.

In [ ]:
union Cantidad
{
    int gramos;
    float kilos;
    int mililitros;
    float litros;
};

Una vez declarada la unión que representará la cantidad necesaria para el ingrediente podemos declarar el registro (**struct**) que usaremos para dichos ingredientes. Cada ingrediente tendrá nombre y cantidad. En este momento surge un pregunta, si la unión puede tener varios campos pero sólo uno de ellos es accesible ¿Cómo vamos a saber cuál debemos leer? &#x1F9D0;. Pues bien, para ello agregaremos un tercer campo que almacene la unidad de medida utilizada para representar la cantidad.

Nuestro registro ingrediente se verá así:

In [ ]:
struct Ingrediente
{
    string nombre;
    Cantidad cantidad;
    int unidadMedida;
};

Para la unidad de medida definiremos un enumerado (el contenedor que nos permite agrupar una colección de constantes). recuerda, **no queremos números mágicos en nuestro programa**.

In [ ]:
enum Unidades
{
    U_GRAMO = 1,
    U_KILO,
    U_MILILITRO,
    U_LITRO
};

Ahora bien, dado que nuestro programa debe manejar un menú, es recomendable tener constantes con nombres significativos para las opciones de dicho menú; con ello ganamos claridad al momento de leer el código y sobre todo hacemos más fácil el proceso de mantenimiento y corrección de errores.

Si por alguna razón debe cambiar la numeración del menú, basta con modificar los datos del enumerado y el resto del programa seguirá funcionando dado que dependerá del contenido de ese enumerado y no de "números mágicos".

In [ ]:
enum Opciones
{
    OPC_SALIR,
    OPC_AGREGAR,
    OPC_CONSULTAR,
    OPC_BUSCAR
};

Antes de comenzar con las implementaciones funcionales de código agregaremos las constantes y variables globales necesarias para el programa.

In [ ]:
//Constante para el total de ingredientes
const int MAX_INGREDIENTE = 10;
//Arreglo de ingredientes
Ingrediente ingredientes[MAX_INGREDIENTE];
//Contador de ingredientes registrados
int contador;

En nuestro programa definiremos un procedimiento para cada una de las opciones de nuestro menú.

Comencemos por el procedimiento para agregar ingredientes. Haremos que dicho procedimiento reciba por referencia el ingrediente que deseamos agregar, o dicho de otra forma, el procedimiento recibirá la referencia del ingrediente al cual le asignaremos valor a sus campos. Recuerda que en un paso por referencia no hacemos copia de la variable, por lo tanto cualquier cambio realizado en la variable recibida se reflejará en la variable que utilizamos al momento de llamar el procedimiento.

In [ ]:
//Recibe un ingrediente por referencia
void registrarIngrediente(Ingrediente& ingrediente)
{
    cout <<"Nombre: ";
    getline(cin, ingrediente.nombre); //Solicitamos el nombre del ingrediente
    do
    {
        //Debemos seleccionar la unidad en que se registrará el ingrediente
        cout <<"¿En qué unidad se registrará el ingrediente? " <<endl
            <<U_GRAMO <<") Gramos" <<endl
           <<U_KILO <<") Kilos" <<endl
          <<U_MILILITRO <<") Mililitros" <<endl
         <<U_LITRO <<") Litros" <<endl
        <<"Selecciona una opción: ";
        cin >>ingrediente.unidadMedida;
    }
    while(ingrediente.unidadMedida < U_GRAMO || ingrediente.unidadMedida > U_LITRO);
    //Solicitaremos el dato mientras sea incorrecto
    
    cout <<"Cantidad: ";
    //Solicitamos la cantidad según la unidad correspondiente
    switch (ingrediente.unidadMedida)
    {
    case U_GRAMO:
        cin >>ingrediente.cantidad.gramos;
        break;
    case U_KILO:
        cin >>ingrediente.cantidad.kilos;
        break;
    case U_MILILITRO:
        cin >>ingrediente.cantidad.mililitros;
        break;
    case U_LITRO:
        cin >>ingrediente.cantidad.litros;
        break;
    default:
        break;
    }
}

A continuación definiremos el método para consultar un ingrediente.

In [ ]:
//Recibe una referencia constante, es decir que a pesar de recibir una referencia
//no puede ser modificada
void consultarIngrediente(const Ingrediente& ingrediente)
{
    cout <<"Nombre: " <<ingrediente.nombre <<endl
        <<"Cantidad: ";
    //La cantidad debe ser consultada según la unidad en que fue guardada
    switch (ingrediente.unidadMedida)
    {
    case U_GRAMO:
        cout <<ingrediente.cantidad.gramos <<"grs" <<endl;
        break;
    case U_KILO:
        cout <<ingrediente.cantidad.kilos <<"kilos" <<endl;
        break;
    case U_MILILITRO:
        cout <<ingrediente.cantidad.mililitros <<"mls" <<endl;
        break;
    case U_LITRO:
        cout <<ingrediente.cantidad.litros <<"lts" <<endl;
        break;
    default:
        break;
    }
}

Para la búsqueda realizaremos búsqueda por coincidencia de patrones sobre el nombre del ingrediente. ¿Qué quiere decir eso? le solicitaremos al usuario que escriba el nombre del ingrediente y buscaremos en el nombre de todas los ingredientes ese patrón, si lo encontramos entonces imprimimos los datos del ingrediente. Por ejemplo si tuviéramos los ingredientes "aceite", "harina" y "mantequilla" y al momento de solicitar el nombre del ingrediente el usuario escribiera "t" se mostraría en pantalla los datos de "aceite" y "mantequilla" porque tienen ese símbolo; si escribiera "ar" entonces sólo mostraría los datos de "harina" porque es el único nombre en el que se cumple el patrón.

In [ ]:
void buscarIngrediente()
{
    string patron;
    int i;
    int coincidencias = 0;

    cout <<"Nombre: ";
    getline(cin, patron);
    for (i = 0; i < contador; ++i)
    {
        //La búsqueda se hace por coincidencia de patrones. Para cada ingrediente
        //registrado se busca el patrón ingresado a partir de la posición cero.
        //La función find() de las cadenas regresa la posición a partir de la cual
        //se encontró el patrón o el tamaño de la cadena en caso de no haberlo
        //encontrado.
        if (ingredientes[i].nombre.find(patron, 0) < ingredientes[i].nombre.size())
        {
            consultarIngrediente(ingredientes[i]);
            cout <<".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-." <<endl;
            ++coincidencias;
        }
    }
    if (coincidencias == 0)
    {
        cout <<"No se encontró el ingrediente" <<endl;
    }
}

A continuación implementamos el menú dentro del cual estarán los llamados a los métodos previamente implementados.

In [ ]:
void menu()
{
    int opc;
    do
    {
        cout <<"                        Menú Ingredientes" <<endl
            <<OPC_AGREGAR <<") Agregar" <<endl
           <<OPC_CONSULTAR <<") Consultar" <<endl
          <<OPC_BUSCAR <<") Buscar" <<endl
         <<OPC_SALIR <<") Salir" <<endl
        <<"Selecciona una opción: ";
        cin >>opc;
        cin.ignore();

        switch (opc)
        {
        case OPC_AGREGAR:
            cout <<"                        Registro de ingrediente" <<endl;
            if (contador < MAX_INGREDIENTE)
            {
                registrarIngrediente(ingredientes[contador]);
                contador++;
            }
            else
            {
                cout <<"No hay espacio para más ingredientes" <<endl;
            }
            break;
        case OPC_CONSULTAR:
            cout <<"                        Ingredientes registrados" <<endl;
            if (contador == 0)
            {
                cout <<"No hay ingredientes registrados" <<endl;
            }
            else
            {
                for (int i = 0; i < contador; ++i)
                {
                    consultarIngrediente(ingredientes[i]);
                    cout <<".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-." <<endl;
                }
            }
            break;
        case OPC_BUSCAR:
            cout <<"                        Búsqueda de Ingredientes" <<endl;
            if (contador == 0)
            {
                cout <<"No hay ingredientes registrados" <<endl;
            }
            else
            {
                buscarIngrediente();
            }
            break;
        case OPC_SALIR:
            cout <<"Nos vemos" <<endl;
            break;
        default:
            cout <<"Opción no válida" <<endl;
            break;
        }
    }
    while(opc != OPC_SALIR);
}

Finalmente en la función principal inicializamos el contador y llamamos al método menú que acabamos de definir.

In [ ]:
int main()
{
    contador = 0;
    menu();

    return 0;
}

Ahora solo resta hacer el llamado a la función principal y probar nuestro sistema y listo coloquemos ingredientes. &#x1f9c2;&#x1f969;&#x1f35e;&#x1f951;&#x1f345;

In [ ]:
main();

Aquí tienes el <a href="https://drive.google.com/file/d/1-lS9uTegYKXfwsmFC8aKwXSlNSkKBLAo/view?usp=sharing">enlace</a> al archivo con el código completo.